In [ ]:
import pandas as pd
import numpy as np

# Toy dataset
data_filtered = pd.DataFrame({
    'amount1': [-100, -50, 100, 50, -30],  # Negative values for buy transactions
    'arb_pnl': [0.02, 0.01, 0.03, 0.04, 0.01],
    'std': [0.1, 0.2, 0.15, 0.05, 0.15]
})

# Parameters for quantile calculations
params = {
    'arb_pnl': {'lower_quantile': 0.25, 'upper_quantile': 0.75},
    'amount1': {'lower_quantile': 0.25, 'upper_quantile': 0.75},
    'std': {'lower_quantile': 0.25, 'upper_quantile': 0.75}
}

# Fees range
f_min, f_max = 0.01, 0.1

# Isolate buy transactions
buy_subset = data_filtered[data_filtered['amount1'] < 0].copy()

features = ['arb_pnl', 'std', 'amount1']
for f in features:
    # Calculate quantile thresholds
    scale_min = buy_subset[f].quantile(params[f]['upper_quantile'])
    scale_max = buy_subset[f].quantile(params[f]['lower_quantile'])
    
    # Compute weight for fee adjustment (simplified example calculation)
    w_buy = np.abs((f_max - f_min) / (scale_max - scale_min))
    
    # Apply fee calculation (simplified as a function of the feature value and weight)
    buy_subset[f'{f}_fee'] = buy_subset[f].apply(lambda x: max(f_min, min(f_max, f_min + w_buy * (x - scale_min))))

# Assume equal importance for simplicity
features_importance_normalized = {'arb_pnl': 0.3, 'std': 0.2, 'amount1': 0.5}

# Apply feature importances as weights
for f in features:
    weight = features_importance_normalized[f]
    buy_subset[f'{f}_weighted_fee'] = buy_subset[f'{f}_fee'] * weight

# Sum weighted fees to get a combined fee and normalize
buy_subset['combined_fee'] = buy_subset[['arb_pnl_weighted_fee', 'std_weighted_fee']].sum(axis=1) / 100

print(buy_subset)#[['arb_pnl', 'std', 'arb_pnl_fee', 'std_fee', 'combined_fee']])
features_importance_normalized
